In [1]:
from Pioneer3.Controllers import PioneerRobot

robot = PioneerRobot()

In [2]:
import cv2
import numpy
import time


def color_blob(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    M = cv2.moments(mask)
    area = M['m00']
    if area > 0:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:
        cx = None
        cy = None
    return area, cx, cy

In [3]:
def is_blob_centered(image):
    area, cx, cy = color_blob(image)
    if area > 0 and cx >= 130 and cx < 145:
        return True
    else:
        return False

In [4]:
def is_blob_close(image, vertical_threshold):
    area, cx, cy = color_blob(image)
    if area > 0 and cy >= vertical_threshold:
        return True
    else:
        return False

In [5]:
def is_obstacle_detected():
    distance = robot.getSonar()
    return distance[3] < 0.45 or distance[4] < 0.45

In [6]:
def line_centroid():
    image = robot.kinect.getColorImage()
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_cyan = numpy.array([140, 25, 40])
    upper_cyan = numpy.array([170, 255, 255])
    mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
    
    mask[0:0, 0:300] = 0
    M = cv2.moments(mask)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    return cx, cy

In [7]:
def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    robot.setSpeed(w_l, w_r)

In [8]:
def follow_line():
    print('Following the line')
    width = 300
    while not is_obstacle_detected():
        cx, cy = line_centroid()
        err = cx - (width/2)
        linear = 0.2
        angular = -0.01*err
        move(linear, angular)
    print('Obstacle detected')

In [9]:
def getWall():
    distance = robot.getSonar()
    leftSide = min(distance[0:3])
    frontSide = min(distance[3:5])
    while frontSide > 0.3:       
        move(0.2,0)
        distance = robot.getSonar()
        leftSide = min(distance[0:3])
        frontSide = min(distance[3:5])
        
    sensor0 = robot.getSonar()[0]
    while sensor0 > 0.45:
        if frontSide < leftSide:
            v = -0.15 * 3
        else:
            v = -0.15
        sensor0 = robot.getSonar()[0]
        move(0,v)
        distance = robot.getSonar()
        leftSide = min(distance[0:3])
        frontSide = min(distance[3:5])
    move(0,0)

In [10]:
def forward():
    distance = robot.getSonar()
    min_iz = min(distance[0:3])
    min_der = min(distance[5:])
    while distance[3] > 1 and distance[4] > 1 and min_iz > 1 and min_der > 1:
        robot.setSpeed(10, 10)
        distance = robot.getSonar()
        min_iz = min(distance[0:3])
        min_der = min(distance[5:])

In [11]:
def is_line_detected():
    image = robot.kinect.getColorImage()
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower_cyan = numpy.array([140, 25, 40])
    upper_cyan = numpy.array([170, 255, 255])
    mask = cv2.inRange(hsv, lower_cyan, upper_cyan)
    mask[0:0, 0:300] = 0
    M = cv2.moments(mask)
    if M['m00'] == 0:
        return False
    return True

In [12]:
def follow_wall():
    print('Following the wall')
    while not is_line_detected():
        distance = robot.getSonar()
        leftSide = min(distance[0:3])
        frontSide = min(distance[3:5])
        vx = 0.2
        v = 0.15
        if frontSide < 0.45:
            vx = -0.1
            v = -0.2 * 4
            
        else:
            if leftSide > 0.45:
                vx  = 0.2*0.5
                v = 0.2
            else:
                if leftSide < 0.45:
                    vx  = 0.2*0.5
                    v = -0.2
        move(vx,v)
    print('Line detected')
    move(0,0)

In [13]:
def getLine():
    cx, cy = line_centroid()
    err = cx - (300 / 2)
    while abs(err) >= 15:
        linear = 0.2
        if err > 0:
            angular = -0.1
        else:
            angular = 0.1
        move(linear, angular)
        cx, cy = line_centroid()
        err = cx - (300 / 2)

In [14]:
def is_blob_centered_target(image):
    area, cx, cy = color_blob(image)
    if area > 0 and cx >= 120 and cx < 130:
        return True
    else:
        return False

In [15]:
def recogerBolas(image):
    time.sleep(2)
    #Recoger bolas
    robot.gripper.open()
    time.sleep(2)
    robot.gripper.down()
    robot.kinect.setTiltPosition(0.05) # tilt down the Kinect    
    
    while not is_blob_centered(image):
        robot.setSpeed(-0.5,0.5)
        image = robot.kinect.getColorImage()
    robot.stop()
    
    while not is_blob_close(image, 155):
        robot.setSpeed(1.0,1.0)
        image = robot.kinect.getColorImage()
    robot.stop()
    
    while not is_blob_centered(image):
        robot.setSpeed(-0.5,0.5)
        image = robot.kinect.getColorImage()
    robot.stop()
    
    robot.kinect.setTiltPosition(-0.35) # tilt down the Kinect
    time.sleep(2)
    


In [16]:
def aproximarYAgarrar(image):
        #Funcion para aproximar y agarrar
    while not is_blob_close(image, 156):
        robot.setSpeed(0.5,0.5)
        image = robot.kinect.getColorImage()
    robot.stop()
    time.sleep(2)
    
    robot.gripper.open()
    time.sleep(2)
    robot.gripper.down()
    time.sleep(2)
    robot.gripper.close()
    time.sleep(2)
    robot.gripper.up()
    time.sleep(2)
    robot.kinect.setTiltPosition(0.05) # tilt down the Kinect
    time.sleep(1)
    

In [17]:
def irAlSitio(image):
    while not is_blob_centered_target(image):
        robot.setSpeed(-0.5,0.5)
        image = robot.kinect.getColorImage()
    robot.stop()
    
    time.sleep(2)

    while not is_blob_close(image, 150):
        robot.setSpeed(1.0,1.0)
        image = robot.kinect.getColorImage()
    robot.stop()
    time.sleep(2)

In [18]:
def dejarBola():
    robot.gripper.down()
    time.sleep(2)
    robot.gripper.open()
    time.sleep(2)
    #Marcha atrás
    robot.setSpeed(-1.0,-1.0)
    time.sleep(10)
    robot.stop()
    #Giro
    robot.setSpeed(-1.0,0)
    time.sleep(3)
    robot.stop()

In [19]:
lower = numpy.array([140, 25, 40])
upper = numpy.array([170, 255, 255])

In [20]:
robot.kinect.setTiltPosition(-0.35) # tilt down the Kinect

In [21]:
def loop():
    try:
        while True:
            follow_line()
            getWall()
            follow_wall()
            getLine()
    except ZeroDivisionError:
        move(0,0)
loop()

Following the line
Obstacle detected
Following the wall
Line detected
Following the line


In [ ]:
veces = 0
image = robot.kinect.getColorImage()
while veces < 5:
    lower = numpy.array([110, 100, 100])
    upper = numpy.array([130, 255, 255])
    recogerBolas(image)
    time.sleep(1)
    aproximarYAgarrar(image)
    time.sleep(1)
    lower = numpy.array([ 50, 100, 100])
    upper = numpy.array([ 70, 255, 255])
    irAlSitio(image)
    time.sleep(1)
    dejarBola()
    veces += 1